# 数组

数组是一连串固定个数的同类型的数据,他们是一段连续的内存空间,数组是最高效也是最简单的数据结构,它的声明方式:

```c
类型 变量名[元素个数];
```
如果是多维数组,则可以是:

```c
类型 变量名[元素个数][元素个数][元素个数]...;
```

数组可以通过下标访问内部元素,下标从0开始计数,c语言中数组的大小也是有限制的,尤其在函数内部定义的数组长度是很有限的.各个编译器不同,但一般还是够用的,但最好不要定义过大,以避免栈溢出

## 多维数组

C语言中的多为数组是从最右边的下标开始变化的,如果把多位数组看作是张量,那么最右边的就是最后面的向量,比如二维数组,右边的下标是向量,左边的下标向量的个数

## 数组初始化

C语言是比较原始的,声明的数组不会自动初始化,必须手动实现,以一个一维数组为例:

```c
int arrayA[5] = {0,0,0,0,0};
```
而多维数组的初始化不过是将其中的元素替换为数组而已

```c
int arrayA[5][2] = {{0,0},{0,0},{0,0},{0,0},{0,0}};
```
如果不初始化而直接访问声明的数组,那么你会看到其中的元素是不知道啥玩意儿

## 访问数组元素

和python中的list,tuple类似,C中访问数组的元素也是使用`[i]`的形式,其实这个操作和指针的偏移量一致

## 数组作为函数参数

+ 一维数组作为函数参数可以不写长度
+ 二维数组作为函数参数可以不写行数,但列数必须写

## 数组越界问题

C语言算是比较原始的,数组的下标如果超过数组本身长度,编译器是可以编译的,也可以正常执行,就是结果会很诡异,因此需要避免.

## 用数组模拟矩阵

很多时候我们希望利用C扩展python的矩阵运算性能,这种时候多维数组就是我们最常接触的工具了


> 例,计算矩阵$\left(\begin{matrix}1 & 2 &3\\4 & 5 & 6\end{matrix}\right)$中最大的元素

In [5]:
%%writefile ../src/C1/section7/matrix_maxelem.cpp
#include <stdio.h>


int main(void){
    int max_elem = 0;
    int matrix[3][2] = {{1,4},{2,5},{3,6}};
    for (int i = 0;i<3;i++){
        for (int j = 0;j<2;j++){
            if (matrix[i][j]>=max_elem) {
                max_elem = matrix[i][j];
            }
        }
    }
    printf("max element is %d\n",max_elem);
    return 0;
}



Overwriting ../src/C1/section7/matrix_maxelem.cpp


In [6]:
!gcc-7 -o ../bin/matrix_maxelem ../src/C1/section7/matrix_maxelem.cpp

In [7]:
!../bin/matrix_maxelem

max element is 6


这是使用原生的数组进行实现,这种形式其实并不方便使用,更好的方式其实是开辟一段一维数组,然后通过封装实现功能.我们后面进行实现

## 数组与指针

很多学过C的人都认为数组和指针实际是一回事,其实不是这么回事,数组只是在一些情况下可以指针等价,但并不是一回事.以下情况下数组和指针是一回事:

+ 表达式中的数组名就是指针

    假如我们声明:`int a[10], *p,i = 2;`
    我们就可以通过以下任何一种方式来访问`a[i]`
    
    + `p = a; p[i];`
    + `p = a; *(p + i);`
    + `p = a + i; *p;`

+ 数组下标作为指针的偏移量

+ 作为函数参数的数组名等同于指针



实际上编译器处理指针和数组的处理方式是不同的:

首先需要注意的是`地址 y`和`地址 y 的内容`之间的区别.因为在大多数编程语言中我们用同一个符号来表示这两个东西,由编译器根据上下文环境判断它的具体含义.

```
x = y;
```

上面的式子就是最简单的赋值语句.其中等号的左边叫做左值,右边叫做右值.

编译器为每个变量分配一个地址作为左值,这个地址在编译时是可知的,而且该变量在运行时一直保存于这个地址.相反的,存储于变量中的值(右值)只有在运行时才可知.如果需要用到变量中存储的值,编译器就发出指令从指定地址读入变量值并将它存于寄存器中.

这里的关键之处在于每个符号的地址在编译时可知,所以,如果编译器需要一个地址(可能还需要加上偏移量)来执行某种操作,它就可以直接进行操作,并不需要增加指令首先取得具体地址,相反,对于指针,必须首先在运行时取得它的当前值,然后才能对它进行解除引用操作.

如果申明为数组`extern char a[]`,那么`a`也就是一个内存地址.数组内的字符可以从这个地址找到.编译器并不需要知道数组有多长,因为它只产生从起始位置起的到偏移量位置的便宜地址.

如果申明为`extern char *p`,那么它将告诉编译器p是一个指针,指向的对象是一个字符,为了取得这个字符,必须得到地址p的内容,把它作为字符的地址并从这个地址中取得这个字符.指针的访问灵活许多,但需要增加一次额外的提取.

如果定义为指针,但以数组方式引用`char *p="asdf";c=p[i]`:
    
那么编译器会了解到有个符号p,地址已知,比如1234,那么运行时他的操作为:
1. 取地址1234中的内容,也就是其中存的内容的地址,比如4321,
2. 取i的值并与p中保存的值相加
3. 取地址[4321+i]这个地址的内容
    
既然把p声明为一个指针,那么不管原先是定义为指针还是数组,都会参照上面的步骤进行操作,但只有当p原来定义为指针时这个方法才是正确的.比如在头文件中申明为`char* p`,但是实现是char p[10];这种时候p[i]取到的实际是个字符,但编译器却把它当作一个指针,因此就会出错.避免这种错误的方法也就是要声明一致.


### 数组和指针的其他区别

指针|数组
---|---
保存数据的地址|保存数据
间接访问数据,首先取得指针的内容,把它作为地址,然后从这个地址提取数据.如果指针有一个下标,就把指针的内容加上这个下标作为地址,从中提取数据|直接访问数据,`a[i]`只是简单的以`a+i`为地址取得数据
通常用于动态数据结构|通常用于存储固定数据且数据类型相同的元素
相关的函数为`malloc`和`free`|隐式分配和删除
通常指向匿名数据|自身即为数据名

定义指针是编译器并不为指针所指向的对象分配空间,它只分配指针本身的空间.


### 特例初始化字符串

在标准C中.初始化指针时所创建的字符串常量被定义为只读,如果试图通过指针修改这个字符串,程序就会出现未定义的行为,字符串常量一般被存放在只读的文本段中以防止被修改.

与指针相反,使用字符串常量初始化数组,那么字符串常量就是可以被修改的

```
char a[]="gooseberry";
strncpy(a,"black",5);
```
结果数组就会被修改为`blackberry`.

## 动态数组

C语言中数组是静态的,一旦编译好了就是固定好了的长度.这就对哪些需要在运行时分配的情况非常不友好,我们只能用一些迂回的方式实现动态数组,具体来说就是使用malloc函数动态分配内存空间的数组.

其原理就是用`calloc`动态的在堆上开辟空间.

